In [1]:
import pandas as pd
import re
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras import layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('csv/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test = pd.read_csv('csv/test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
tf.random.set_seed(2)

#### Lectura BERT

In [ ]:
bert_train = np.loadtxt("train_embedding_file_bert_lemm.txt").reshape(len(train.index),len_embedding)
bert_train.shape

In [ ]:
bert_test = np.loadtxt("test_embedding_file_bert_lemm.txt").reshape(len(test.index),len_embedding)
bert_test.shape

#### Lectura ELMo

In [ ]:
elmo_train = np.loadtxt("train_embedding_file_elmo_lemm.txt").reshape(len(train.index),len_embedding)
elmo_train.shape

In [ ]:
elmo_test = np.loadtxt("test_embedding_file_elmo_lemm.txt").reshape(len(test.index),len_embedding)
elmo_test.shape

#### Lectura W2V

In [ ]:
w2v_train = np.loadtxt("train_embedding_file_w2v(norm)_lemm.txt").reshape(len(train.index),len_embedding)
w2v_train.shape

In [ ]:
w2v_test = np.loadtxt("test_embedding_file_w2v(norm)_lemm.txt").reshape(len(test.index),len_embedding)
w2v_test.shape

#### Lectura Glove

In [ ]:
glove_train = np.loadtxt("train_embedding_file_glove_lemm.txt").reshape(len(train.index),len_embedding)
glove_train.shape

In [ ]:
glove_test = np.loadtxt("test_embedding_file_glove_lemm.txt").reshape(len(test.index),len_embedding)
glove_test.shape

# BERT

In [6]:
len_embedding = 768

In [9]:
X, y = bert_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [10]:
model = Sequential()
model.add(layers.Dense(384, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(192, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 384)               295296    
_________________________________________________________________
dropout (Dropout)            (None, 384)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 192)               73920     
_________________________________________________________________
dropout_1 (Dropout)          (None, 192)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 193       
Total params: 369,409
Trainable params: 369,409
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 5s 7ms/step - loss: 0.5686 - accuracy: 0.7097 - val_loss: 0.4623 - val_accuracy: 0.7835
Epoch 2/10
686/686 [==============================] - 4s 6ms/step - loss: 0.4858 - accuracy: 0.7762 - val_loss: 0.4312 - val_accuracy: 0.8097
Epoch 3/10
686/686 [==============================] - 4s 6ms/step - loss: 0.4583 - accuracy: 0.7951 - val_loss: 0.4230 - val_accuracy: 0.8163
Epoch 4/10
686/686 [==============================] - 4s 6ms/step - loss: 0.4423 - accuracy: 0.8028 - val_loss: 0.4137 - val_accuracy: 0.8150
Epoch 5/10
686/686 [==============================] - 4s 6ms/step - loss: 0.4317 - accuracy: 0.8114 - val_loss: 0.4243 - val_accuracy: 0.8058
Epoch 6/10
686/686 [==============================] - 5s 7ms/step - loss: 0.4217 - accuracy: 0.8136 - val_loss: 0.4213 - val_accuracy: 0.8045
Epoch 7/10
686/686 [==============================] - 5s 7ms/step - loss: 0.4142 - accuracy: 0.8168 - val_loss: 0.4091 - val_accuracy: 0.8150
Epoch 

In [13]:
test_pred = model.predict(bert_test)
test_pred_int = test_pred.round().astype('int')

In [14]:
test['bert_net'] = test_pred
test = test['bert_net'].to_frame()
test.head()

,bert_net
0,0.876963
1,0.777118
2,0.773498
3,0.957246
4,0.921719


In [15]:
test.to_csv('csv/test_red_bert_lemm.csv')

In [16]:
train_pred = model.predict(bert_train)
train_pred_int = train_pred.round().astype('int')

In [17]:
train['bert_net'] = train_pred
train = train['bert_net'].to_frame()
train.to_csv('csv/train_red_bert_lemm.csv')

# ELMo

In [18]:
len_embedding = 1024

In [21]:
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
X, y = elmo_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [22]:
model = Sequential()
model.add(layers.Dense(512, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(256, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [23]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 656,385
Trainable params: 656,385
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 8s 11ms/step - loss: 0.5418 - accuracy: 0.7400 - val_loss: 0.4699 - val_accuracy: 0.8097
Epoch 2/10
686/686 [==============================] - 8s 11ms/step - loss: 0.4633 - accuracy: 0.7923 - val_loss: 0.4552 - val_accuracy: 0.8058
Epoch 3/10
686/686 [==============================] - 8s 11ms/step - loss: 0.4486 - accuracy: 0.7954 - val_loss: 0.4469 - val_accuracy: 0.8163
Epoch 4/10
686/686 [==============================] - 8s 12ms/step - loss: 0.4350 - accuracy: 0.8065 - val_loss: 0.4400 - val_accuracy: 0.8202
Epoch 5/10
686/686 [==============================] - 8s 12ms/step - loss: 0.4234 - accuracy: 0.8124 - val_loss: 0.4437 - val_accuracy: 0.7966
Epoch 6/10
686/686 [==============================] - 8s 12ms/step - loss: 0.4106 - accuracy: 0.8186 - val_loss: 0.4450 - val_accuracy: 0.7966
Epoch 7/10
686/686 [==============================] - 8s 11ms/step - loss: 0.4039 - accuracy: 0.8229 - val_loss: 0.4333 - val_accuracy: 0.8136

In [25]:
test_pred = model.predict(elmo_test)
test_pred_int = test_pred.round().astype('int')

In [26]:
test['elmo_net'] = test_pred
test = test['elmo_net'].to_frame()
test.head()

,elmo_net
0,0.423664
1,0.666756
2,0.726632
3,0.637715
4,0.772717


In [27]:
test.to_csv('csv/test_red_elmo_lemm.csv')

In [28]:
train_pred = model.predict(elmo_train)
train_pred_int = train_pred.round().astype('int')

In [29]:
train['elmo_net'] = train_pred
train = train['elmo_net'].to_frame()
train.to_csv('csv/train_red_elmo_lemm.csv')

# W2V

In [30]:
len_embedding = 300

In [33]:
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
X, y = w2v_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [34]:
model = Sequential()
model.add(layers.Dense(150, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(75, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [35]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 150)               45150     
_________________________________________________________________
dropout_4 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 75)                11325     
_________________________________________________________________
dropout_5 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 76        
Total params: 56,551
Trainable params: 56,551
Non-trainable params: 0
_________________________________________________________________


In [36]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 2s 2ms/step - loss: 0.6527 - accuracy: 0.6011 - val_loss: 0.6065 - val_accuracy: 0.6942
Epoch 2/10
686/686 [==============================] - 2s 2ms/step - loss: 0.5784 - accuracy: 0.7402 - val_loss: 0.5288 - val_accuracy: 0.7874
Epoch 3/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5219 - accuracy: 0.7689 - val_loss: 0.4917 - val_accuracy: 0.7927
Epoch 4/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5025 - accuracy: 0.7774 - val_loss: 0.4793 - val_accuracy: 0.8031
Epoch 5/10
686/686 [==============================] - 1s 2ms/step - loss: 0.4819 - accuracy: 0.7878 - val_loss: 0.4704 - val_accuracy: 0.7979
Epoch 6/10
686/686 [==============================] - 2s 2ms/step - loss: 0.4726 - accuracy: 0.7905 - val_loss: 0.4629 - val_accuracy: 0.8071
Epoch 7/10
686/686 [==============================] - 2s 2ms/step - loss: 0.4632 - accuracy: 0.7952 - val_loss: 0.4572 - val_accuracy: 0.8045
Epoch 

In [37]:
test_pred = model.predict(w2v_test)
test_pred_int = test_pred.round().astype('int')

In [38]:
test['w2v_net'] = test_pred
test = test['w2v_net'].to_frame()
test.head()

,w2v_net
0,0.810528
1,0.626968
2,0.847257
3,0.813005
4,0.891987


In [39]:
test.to_csv('csv/test_red_w2v_lemm.csv')

In [40]:
train_pred = model.predict(w2v_train)
train_pred_int = train_pred.round().astype('int')

In [41]:
train['w2v_net'] = train_pred
train = train['w2v_net'].to_frame()
train.to_csv('csv/train_red_w2v_lemm.csv')

# GLOve

In [42]:
len_embedding = 200

In [45]:
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
X, y = glove_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [46]:
model = Sequential()
model.add(layers.Dense(100, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(50, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [47]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 25,201
Trainable params: 25,201
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 2s 3ms/step - loss: 0.9537 - accuracy: 0.5983 - val_loss: 0.5841 - val_accuracy: 0.7467
Epoch 2/10
686/686 [==============================] - 2s 3ms/step - loss: 0.7410 - accuracy: 0.6647 - val_loss: 0.5272 - val_accuracy: 0.7638
Epoch 3/10
686/686 [==============================] - 1s 2ms/step - loss: 0.6361 - accuracy: 0.7002 - val_loss: 0.5060 - val_accuracy: 0.7782
Epoch 4/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5954 - accuracy: 0.7200 - val_loss: 0.4874 - val_accuracy: 0.7808
Epoch 5/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5729 - accuracy: 0.7288 - val_loss: 0.4815 - val_accuracy: 0.7756
Epoch 6/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5439 - accuracy: 0.7412 - val_loss: 0.4747 - val_accuracy: 0.7822
Epoch 7/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5259 - accuracy: 0.7519 - val_loss: 0.4688 - val_accuracy: 0.7861
Epoch 

In [49]:
test_pred = model.predict(glove_test)
test_pred_int = test_pred.round().astype('int')

In [50]:
test['glove_net'] = test_pred
test = test['glove_net'].to_frame()
test.head()

,glove_net
0,0.635642
1,0.703766
2,0.817606
3,0.757318
4,0.865400


In [51]:
test.to_csv('csv/test_red_glove_lemm.csv')

In [52]:
train_pred = model.predict(glove_train)
train_pred_int = train_pred.round().astype('int')

In [53]:
train['glove_net'] = train_pred
train = train['glove_net'].to_frame()
train.to_csv('csv/train_red_glove_lemm.csv')